In [1]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
### 使用 networkx 包中的函数 LFR_benchmark_graph 生成随机图
import networkx as nx
from networkx.generators.community import LFR_benchmark_graph

n = 1000
tau1 = 2  # Power-law exponent for the degree distribution
tau2 = 1.1 # Power-law exponent for the community size distribution 
            #S hould be >1
mu = 0.05 # Mixing parameter
avg_deg = 25 # Average Degree
max_deg = 100 # Max Degree
min_commu = 80 # Min Community Size
max_commu = 100 # Max Community Size

G = LFR_benchmark_graph(
    n, tau1, tau2, mu, average_degree=avg_deg, max_degree=max_deg, min_community=min_commu, max_community=max_commu, 
    seed=7
)
### 去掉 G 中的重边和自环 
G = nx.Graph(G) # Remove multi-edges

selfloop_edges = list(nx.selfloop_edges(G)) # a list of self loops

G.remove_edges_from(selfloop_edges) # Remove self-loops

In [3]:
import numpy as np
intrinsic_communities = {frozenset(G.nodes[v]["community"]) for v in G}
intrinsic_membership = np.empty(G.number_of_nodes(), dtype=int)
for node in range(G.number_of_nodes()):
    for index, inner_set in enumerate(intrinsic_communities):
        if node in inner_set:
            intrinsic_membership[node] = index
            break

In [4]:
G.remove_node(2)

In [5]:
G.nodes()

NodeView((0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220

In [6]:
idx = [True] *(G.number_of_nodes()+1)
idx[2]=False
intrinsic_membership=intrinsic_membership[idx]

In [7]:
len(np.unique(intrinsic_membership))

11

In [8]:
D=12
### 1 Hope 方法
from auxpack.evaluate_embd702temp import evaluate_embd as EEE
from gem.embedding.hope import HOPE    
hope_model = HOPE(d=D, beta=0.01) 
# A higher value of beta places more emphasis on capturing higher-order proximities
embd = hope_model.learn_embedding(graph=G, is_weighted=False, no_python=True)
defen = EEE(11, intrinsic_membership, embd)
print("NMI:", defen)

NMI: 0.7899093216796161


In [10]:
D=89
### 2 Laplacian 方法
from gem.embedding.lap import LaplacianEigenmaps

lap_model = LaplacianEigenmaps(d=D)
embd = lap_model.learn_embedding(graph=G, is_weighted=False, no_python=True)
defen = EEE(11, intrinsic_membership, embd)
print("NMI:", defen)

NMI: 0.7757197101483935


In [95]:
%%time
from karateclub import MNMF
D=15

K = max(intrinsic_membership)+1


# Create an instance of the MNMF model
MNMF_model = MNMF(dimensions = D, clusters = K, lambd = 0.2, 
             alpha = 0.05, beta = 0.05, iterations = 100, 
             lower_control = 1e-15, eta = 5.0, seed = 42)

# Fit the model to the graph
MNMF_model.fit(H)

# Obtain the graph embeddings
embd = MNMF_model.get_embedding()

defen = EEE(11, intrinsic_membership, embd)
print("NMI:", defen)

NMI: 1.0
CPU times: user 16.7 s, sys: 13.2 s, total: 29.9 s
Wall time: 1.43 s


In [ ]:
#LLE 方法可以

In [12]:
from node2vec import Node2Vec
import numpy as np

nodes = [str(i) for i in list(G.nodes())]

# Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
node2vec_model = Node2Vec(G, dimensions=5, walk_length=16, num_walks=8, workers=32, quiet=True) #, temp_folder='test' # Use temp_folder for big graphs
# Embed nodes 
node2vec_fit = node2vec_model.fit(window=10, min_count=1, batch_words=16192)  
# Any keywords acceptable by gensim.Word2Vec can be passed, `dimensions` and `workers` are automatically passed 
# (from the Node2Vec constructor)
embd = np.array([node2vec_fit.wv[node] for node in nodes])
defen = EEE(11, intrinsic_membership, embd)
print("NMI:", defen)

NMI: 1.0


In [53]:
#from karateclub import DeepWalk

model = DeepWalk(dimensions=3, walk_length=16, window_size=10)
model.fit(G)
embd = model.get_embedding()
defen = EEE(11, intrinsic_membership, embd)
print("NMI:", defen)

NMI: 0.8146574950869895


In [54]:
from ge import LINE
for D in range(2,50,2):
    model = LINE(G,embedding_size=D,order='first');
    model.train(batch_size=8192,epochs=50,verbose=0);# train model
    LINE_embd = model.get_embeddings();# get embedding vectors

    embd = list(LINE_embd.values())
    defen = EEE(11, intrinsic_membership, embd)
    print("NMI:", defen)

NMI: 0.6693079275527246
NMI: 0.942077257087706
NMI: 0.9921851055453348
NMI: 0.9976925106779044
NMI: 0.9999999999999998
NMI: 1.0
NMI: 1.0000000000000002
NMI: 0.9999999999999998
NMI: 1.0
NMI: 1.0
NMI: 1.0
NMI: 1.0
NMI: 0.9999999999999998
NMI: 1.0
NMI: 0.9999999999999998
NMI: 1.0
NMI: 1.0
NMI: 1.0
NMI: 1.0
NMI: 1.0
NMI: 1.0
NMI: 1.0
NMI: 0.9999999999999998
NMI: 1.0


In [56]:
import random
import warnings
from typing import List
import re
import numpy as np
import networkx as nx
from gensim.models.word2vec import Word2Vec
from karateclub.utils.walker import RandomWalker


class DeepWalk:
    r"""An implementation of `"DeepWalk" <https://arxiv.org/abs/1403.6652>`_
    from the KDD '14 paper "DeepWalk: Online Learning of Social Representations".
    The procedure uses random walks to approximate the pointwise mutual information
    matrix obtained by pooling normalized adjacency matrix powers. This matrix
    is decomposed by an approximate factorization technique.

    Args:
        walk_number (int): Number of random walks. Default is 10.
        walk_length (int): Length of random walks. Default is 80.
        dimensions (int): Dimensionality of embedding. Default is 128.
        workers (int): Number of cores. Default is 4.
        window_size (int): Matrix power order. Default is 5.
        epochs (int): Number of epochs. Default is 1.
        use_hierarchical_softmax (bool): Whether to use hierarchical softmax or negative sampling to train the model. Default is True.
        number_of_negative_samples (int): Number of negative nodes to sample (usually between 5-20). If set to 0, no negative sampling is used. Default is 5.
        learning_rate (float): HogWild! learning rate. Default is 0.05.
        min_count (int): Minimal count of node occurrences. Default is 1.
        seed (int): Random seed value. Default is 42.
    """

    def __init__(
        self,
        walk_number: int = 10,
        walk_length: int = 80,
        dimensions: int = 128,
        workers: int = 4,
        window_size: int = 5,
        epochs: int = 1,
        use_hierarchical_softmax: bool = True,
        number_of_negative_samples: int = 5,
        learning_rate: float = 0.05,
        min_count: int = 1,
        seed: int = 42,
    ):
        self.walk_number = walk_number
        self.walk_length = walk_length
        self.dimensions = dimensions
        self.workers = workers
        self.window_size = window_size
        self.epochs = epochs
        self.use_hierarchical_softmax = use_hierarchical_softmax
        self.number_of_negative_samples = number_of_negative_samples
        self.learning_rate = learning_rate
        self.min_count = min_count
        self.seed = seed

    def fit(self, graph: nx.classes.graph.Graph):
        """
        Fitting a DeepWalk model.

        Arg types:
            * **graph** *(NetworkX graph)* - The graph to be embedded.
        """
        self._set_seed()
        graph = self._check_graph(graph)
        walker = RandomWalker(self.walk_length, self.walk_number)
        walker.do_walks(graph)

        model = Word2Vec(
            walker.walks,
            hs=1 if self.use_hierarchical_softmax else 0,
            negative=self.number_of_negative_samples,
            alpha=self.learning_rate,
            epochs=self.epochs,
            vector_size=self.dimensions,
            window=self.window_size,
            min_count=self.min_count,
            workers=self.workers,
            seed=self.seed,
        )

        num_of_nodes = graph.number_of_nodes()
        self._embedding = [model.wv[str(n)] for n in range(num_of_nodes)]

    def get_embedding(self) -> np.array:
        r"""Getting the node embedding.

        Return types:
            * **embedding** *(Numpy array)* - The embedding of nodes.
        """
        return np.array(self._embedding)

    def get_params(self):
        """Get parameter dictionary for this estimator.."""
        rx = re.compile(r'^\_')
        params = self.__dict__
        params = {key: params[key] for key in params if not rx.search(key)}
        return params

    def _set_seed(self):
        """Creating the initial random seed."""
        random.seed(self.seed)
        np.random.seed(self.seed)

    @staticmethod
    def _ensure_walk_traversal_conditions(graph: nx.classes.graph.Graph) -> nx.classes.graph.Graph:
        """Ensure walk traversal conditions."""
        # We create a copy of the graph
        graph = graph.copy()
        for node_index in list(graph.nodes()):
            if not graph.has_edge(node_index, node_index):
                # And we add the missing edges
                # for filling the main diagonal
                graph.add_edges_from(
                    [(index, index) for index in list(graph.nodes()) if not graph.has_edge(index, index)]
                )
                break

        return graph

    @staticmethod
    def _check_indexing(graph: nx.classes.graph.Graph):
        """Checking the consecutive numeric indexing."""
        numeric_indices = [index for index in range(graph.number_of_nodes())]
        node_indices = sorted([node for node in graph.nodes()])

        assert numeric_indices == node_indices, "The node indexing is wrong."

    def _check_graph(self, graph: nx.classes.graph.Graph) -> nx.classes.graph.Graph:
        """Check the Karate Club assumptions about the graph."""
        #self._check_indexing(graph)
        graph = self._ensure_walk_traversal_conditions(graph)

        return graph

    def _check_graphs(self, graphs: List[nx.classes.graph.Graph]):
        """Check the Karate Club assumptions for a list of graphs."""
        graphs = [self._check_graph(graph) for graph in graphs]

        return graphs

In [80]:
import random
import warnings
from typing import List
from tqdm.auto import trange
import re

import numpy as np
import networkx as nx
from typing import Dict
from scipy.sparse import coo_matrix



class MNMF:
    r"""An implementation of `"M-NMF" <https://aaai.org/ocs/index.php/AAAI/AAAI17/paper/view/14589/13763>`_
    from the AAAI '17 paper "Community Preserving Network Embedding".
    The procedure uses joint non-negative matrix factorization with modularity
    based regularization in order to learn a cluster membership distribution
    over nodes. The method can be used in an overlapping and non-overlapping way.

    Args:
        dimensions (int): Number of dimensions. Default is 128.
        clusters (int): Number of clusters. Default is 10.
        lambd (float): KKT penalty. Default is 0.2
        alpha (float): Clustering penalty. Default is 0.05.
        beta (float): Modularity regularization penalty. Default is 0.05.
        iterations (int): Number of power iterations. Default is 200.
        lower_control (float): Floating point overflow control. Default is 10**-15.
        eta (float): Similarity mixing parameter. Default is 5.0.
        seed (int): Random seed value. Default is 42.
    """

    def __init__(
        self,
        dimensions: int = 128,
        clusters: int = 10,
        lambd: float = 0.2,
        alpha: float = 0.05,
        beta: float = 0.05,
        iterations: int = 200,
        lower_control: float = 10**-15,
        eta: float = 5.0,
        seed: int = 42,
    ):

        self.dimensions = dimensions
        self.clusters = clusters
        self.lambd = lambd
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.lower_control = lower_control
        self.eta = eta
        self.seed = seed

   def _modularity_generator(self):
    """Calculating the sparse modularity matrix."""
    node_indices = list(self._graph.nodes())
    node_mapping = {node: index for index, node in enumerate(node_indices)}
    inv_node_mapping = {index: node for node, index in node_mapping.items()}
    e_count = self._graph.number_of_edges()
    n_count = self._graph.number_of_nodes()
    modularity_mat_shape = (n_count, n_count)
    indices_1 = np.array(
        [node_mapping[edge[0]] for edge in self._graph.edges()]
        + [node_mapping[edge[1]] for edge in self._graph.edges()]
    )
    indices_2 = np.array(
        [node_mapping[edge[1]] for edge in self._graph.edges()]
        + [node_mapping[edge[0]] for edge in self._graph.edges()]
    )
    scores = [
        float(self._graph.degree(inv_node_mapping[e[0]]) * self._graph.degree(inv_node_mapping[e[1]])) / (2 * e_count)
        for e in self._graph.edges()
    ]
    scores = scores + [
        float(self._graph.degree(inv_node_mapping[e[1]]) * self._graph.degree(inv_node_mapping[e[0]])) / (2 * e_count)
        for e in self._graph.edges()
    ]
    mod_matrix = coo_matrix(
        (scores, (indices_1, indices_2)), shape=modularity_mat_shape
    )
    return mod_matrix


    def _setup_matrices(self):
        """Creating parameter matrices and target matrices."""
        self._number_of_nodes = nx.number_of_nodes(self._graph)
        self._M = np.random.uniform(0, 1, (self._number_of_nodes, self.dimensions))
        self._U = np.random.uniform(0, 1, (self._number_of_nodes, self.dimensions))
        self._H = np.random.uniform(0, 1, (self._number_of_nodes, self.clusters))
        self._C = np.random.uniform(0, 1, (self.clusters, self.dimensions))
        self._B1 = nx.adjacency_matrix(
            self._graph, nodelist=list(self._graph.nodes())
        )
        self._B2 = self._modularity_generator()
        self._X = np.transpose(self._U)
        overlaps = self._B1.dot(self._B1)
        self._S = self._B1 + self.eta * self._B1 * (overlaps)

    def _update_M(self):
        """Update matrix M."""
        enum = self._S.dot(self._U)
        denom = np.dot(self._M, np.dot(np.transpose(self._U), self._U))
        denom[denom < self.lower_control] = self.lower_control
        self._M = np.multiply(self._M, enum / denom)
        row_sums = self._M.sum(axis=1)
        self._M = self._M / row_sums[:, np.newaxis]

    def _update_U(self):
        """Update matrix U."""
        enum = self._S.dot(self._M) + self.alpha * np.dot(self._H, self._C)
        denom = np.dot(
            self._U,
            np.dot(np.transpose(self._M), self._M)
            + self.alpha * np.dot(np.transpose(self._C), self._C),
        )
        denom[denom < self.lower_control] = self.lower_control
        self._U = np.multiply(self._U, enum / denom)
        row_sums = self._U.sum(axis=1)
        self._U = self._U / row_sums[:, np.newaxis]

    def _update_C(self):
        """Update matrix C."""
        enum = np.dot(np.transpose(self._H), self._U)
        denom = np.dot(self._C, np.dot(np.transpose(self._U), self._U))
        denom[denom < self.lower_control] = self.lower_control
        frac = enum / denom
        self._C = np.multiply(self._C, frac)
        row_sums = self._C.sum(axis=1)
        self._C = self._C / row_sums[:, np.newaxis]

    def _update_H(self):
        """Update matrix H."""
        B1H = self._B1.dot(self._H)
        B2H = self._B2.dot(self._H)
        HHH = np.dot(self._H, (np.dot(np.transpose(self._H), self._H)))
        UC = np.dot(self._U, np.transpose(self._C))
        rooted = np.square(2 * self.beta * B2H) + np.multiply(
            16 * self.lambd * HHH,
            (
                2 * self.beta * B1H
                + 2 * self.alpha * UC
                + (4 * self.lambd - 2 * self.alpha) * self._H
            ),
        )
        rooted[rooted < 0] = 0
        sqroot_1 = np.sqrt(rooted)
        enum = -2 * self.beta * B2H + sqroot_1
        denom = 8 * self.lambd * HHH
        denom[denom < self.lower_control] = self.lower_control
        rooted = enum / denom
        rooted[rooted < 0] = 0
        sqroot_2 = np.sqrt(rooted)
        self._H = np.multiply(self._H, sqroot_2)
        row_sums = self._H.sum(axis=1)
        self._H = self._H / row_sums[:, np.newaxis]

    def get_memberships(self) -> Dict[int, int]:
        r"""Getting the cluster membership of nodes.

        Return types:
            * **memberships** *(dict)* - Node cluster memberships.
        """
        indices = np.argmax(self._H, axis=1)
        memberships = {i: membership for i, membership in enumerate(indices)}
        return memberships

    def get_embedding(self) -> np.array:
        r"""Getting the node embedding.

        Return types:
            * **embedding** *(Numpy array)* - The embedding of nodes.
        """
        embedding = self._U
        return embedding

    def get_cluster_centers(self) -> np.array:
        r"""Getting the node embedding.

        Return types:
            * **centers** *(Numpy array)* - The cluster centers.
        """
        centers = self._C
        return centers

    def fit(self, graph: nx.classes.graph.Graph):
        """
        Fitting an M-NMF clustering model.

        Arg types:
            * **graph** *(NetworkX graph)* - The graph to be clustered.
        """
        self._set_seed()
        graph = self._check_graph(graph)
        self._graph = graph
        self._setup_matrices()
        for _ in range(self.iterations):
            self._update_M()
            self._update_U()
            self._update_C()
            self._update_H()

    def get_params(self):
        """Get parameter dictionary for this estimator.."""
        rx = re.compile(r'^\_')
        params = self.__dict__
        params = {key: params[key] for key in params if not rx.search(key)}
        return params

    def _set_seed(self):
        """Creating the initial random seed."""
        random.seed(self.seed)
        np.random.seed(self.seed)

    @staticmethod
    def _ensure_walk_traversal_conditions(graph: nx.classes.graph.Graph) -> nx.classes.graph.Graph:
        """Ensure walk traversal conditions."""
        # We create a copy of the graph
        graph = graph.copy()
        for node_index in list(graph.nodes()):
            if not graph.has_edge(node_index, node_index):
                # And we add the missing edges
                # for filling the main diagonal
                graph.add_edges_from(
                    [(index, index) for index in list(graph.nodes()) if not graph.has_edge(index, index)]
                )
                break

        return graph

    @staticmethod
    def _check_indexing(graph: nx.classes.graph.Graph):
        """Checking the consecutive numeric indexing."""
        numeric_indices = [index for index in range(graph.number_of_nodes())]
        node_indices = sorted([node for node in graph.nodes()])

        assert numeric_indices == node_indices, "The node indexing is wrong."

    def _check_graph(self, graph: nx.classes.graph.Graph) -> nx.classes.graph.Graph:
        """Check the Karate Club assumptions about the graph."""
        #self._check_indexing(graph)
        graph = self._ensure_walk_traversal_conditions(graph)

        return graph

    def _check_graphs(self, graphs: List[nx.classes.graph.Graph]):
        """Check the Karate Club assumptions for a list of graphs."""
        graphs = [self._check_graph(graph) for graph in graphs]

        return graphs

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 56)

In [57]:
G0 = LFR_benchmark_graph(
    n, tau1, tau2, mu, average_degree=avg_deg, max_degree=max_deg, min_community=min_commu, max_community=max_commu, 
    seed=7
)
### 去掉 G 中的重边和自环 
G0 = nx.Graph(G0) # Remove multi-edges

selfloop_edges = list(nx.selfloop_edges(G0)) # a list of self loops

G0.remove_edges_from(selfloop_edges) # Remove self-loops

import numpy as np
intrinsic_communities0 = {frozenset(G0.nodes[v]["community"]) for v in G}
intrinsic_membership0 = np.empty(G0.number_of_nodes(), dtype=int)
for node in range(G0.number_of_nodes()):
    for index, inner_set in enumerate(intrinsic_communities0):
        if node in inner_set:
            intrinsic_membership0[node] = index
            break

In [72]:
model = DeepWalk(dimensions=4, walk_length=16, window_size=10)
model.fit(G0)
embd = model.get_embedding()
defen = EEE(11, intrinsic_membership0, embd)
print("NMI:", defen)

NMI: 0.9881605067085244


In [81]:
D=20

K = max(intrinsic_membership)+1


# Create an instance of the MNMF model
MNMF_model = MNMF(dimensions = D, clusters = K, lambd = 0.2, 
             alpha = 0.05, beta = 0.05, iterations = 100, 
             lower_control = 1e-15, eta = 5.0, seed = 42)

# Fit the model to the graph
MNMF_model.fit(G)

# Obtain the graph embeddings
embd = MNMF_model.get_embedding()

ValueError: row index exceeds matrix dimensions

In [83]:
H = nx.relabel.convert_node_labels_to_integers(G)

In [85]:
H.nodes()

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 